<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE4-Sensor-Pretraining_Hourly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Simple Sensor Pre-Training System
=================================

Trains LSTM Autoencoders for each sensor and saves models with baseline statistics.

Usage:
    python sensor_pretraining.py
"""

import numpy as np
import os
import pickle
from datetime import datetime
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


def build_lstm_autoencoder(window_length: int, latent_dim: int = 32) -> Model:
    """Build LSTM Autoencoder."""
    inputs = Input(shape=(window_length, 1))

    # Encoder
    encoded = LSTM(latent_dim, activation='relu', return_sequences=False)(inputs)

    # Decoder
    decoded = RepeatVector(window_length)(encoded)
    decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
    outputs = TimeDistributed(Dense(1, activation='linear'))(decoded)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model


def train_sensor_model(sensor_data, sensor_id, base_path, window_length):
    """Train model for one sensor."""

    print(f"\nTraining sensor {sensor_id}...")
    print(f"Data shape: {sensor_data.shape}")

    # Split data
    n_samples = len(sensor_data)
    n_train = int(0.8 * n_samples)

    X_train = sensor_data[:n_train]
    X_val = sensor_data[n_train:]

    print(f"Train: {len(X_train)}, Val: {len(X_val)}")

    # Create directories
    sensor_dir = os.path.join(base_path, 'sensor', 'model')
    checkpoint_dir = os.path.join(sensor_dir, 'checkpoints')
    os.makedirs(sensor_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Build model
    model = build_lstm_autoencoder(window_length)

    # Setup callbacks
    checkpoint_path = os.path.join(checkpoint_dir, f'sensor_{sensor_id}_best.h5')

    callbacks = [
        # Save best model during training
        ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            verbose=1
        ),
        # Early stopping
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        # Reduce learning rate on plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ]

    # Train model
    print("Starting training with checkpointing...")
    history = model.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )

    print(f"Training completed. Best model saved to: {checkpoint_path}")

    # Compute baseline reconstruction errors on validation set
    print("Computing baseline errors...")
    val_predictions = model.predict(X_val, verbose=0)
    baseline_errors = []

    for i in range(len(X_val)):
        error = mean_squared_error(X_val[i].flatten(), val_predictions[i].flatten())
        baseline_errors.append(error)

    baseline_stats = {
        'mean': float(np.mean(baseline_errors)),
        'std': float(np.std(baseline_errors)) + 1e-8,
        'q95': float(np.percentile(baseline_errors, 95)),
        'q99': float(np.percentile(baseline_errors, 99)),
        'baseline_errors': baseline_errors  # Store for drift detection
    }

    # Save final model and metadata
    model_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_model.h5')
    metadata_path = os.path.join(sensor_dir, f'sensor_{sensor_id}_metadata.pkl')

    model.save(model_path)

    metadata = {
        'sensor_id': sensor_id,
        'window_length': window_length,
        'baseline_stats': baseline_stats,
        'trained_at': datetime.now(),
        'epochs_trained': len(history.history['loss']),
        'final_val_loss': float(history.history['val_loss'][-1]),
        'checkpoint_path': checkpoint_path
    }

    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)

    print(f"✅ Sensor {sensor_id} completed")
    print(f"   Final model: {model_path}")
    print(f"   Best checkpoint: {checkpoint_path}")
    print(f"   Baseline error: {baseline_stats['mean']:.6f} ± {baseline_stats['std']:.6f}")

    return baseline_stats


def main():
    """Main training function."""

    # Your exact paths
    data_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy'
    base_path = r'/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'

    print("🚀 Simple Sensor Pre-Training")
    print("=" * 40)

    # Load data
    print("Loading data...")
    data = np.load(data_path)
    print(f"Original data shape: {data.shape}")

    # Remove last 1000 samples (holdout)
    training_data = data[:-1000]
    print(f"Training data shape: {training_data.shape}")

    batch_size, window_length, num_sensors = training_data.shape
    print(f"Will train {num_sensors} sensors")

    # Train each sensor
    print(f"\n🏋️ Training {num_sensors} LSTM Autoencoders...")

    results = {}
    successful = 0

    for sensor_id in range(num_sensors):
        try:
            # Extract data for this sensor: [batch, timestep, 1]
            sensor_data = training_data[:, :, sensor_id:sensor_id+1]  # Keep feature dimension

            baseline_stats = train_sensor_model(sensor_data, sensor_id, base_path, window_length)
            results[sensor_id] = {'success': True, 'baseline_stats': baseline_stats}
            successful += 1

        except Exception as e:
            print(f"❌ Failed training sensor {sensor_id}: {e}")
            results[sensor_id] = {'success': False, 'error': str(e)}

    # Final summary
    print(f"\n📊 TRAINING SUMMARY")
    print("=" * 40)
    print(f"✅ Successful: {successful}/{num_sensors}")
    print(f"💾 Models saved to: {base_path}/sensor/model/")

    if successful > 0:
        print(f"\n🏆 Successfully trained sensors:")
        for sensor_id, result in results.items():
            if result['success']:
                baseline = result['baseline_stats']
                print(f"  Sensor {sensor_id}: baseline error {baseline['mean']:.6f}")

    # Save training summary
    summary_path = os.path.join(base_path, 'sensor', 'model', 'training_summary.pkl')
    with open(summary_path, 'wb') as f:
        pickle.dump({
            'results': results,
            'training_data_shape': training_data.shape,
            'successful_sensors': successful,
            'timestamp': datetime.now()
        }, f)

    print(f"\n💾 Summary saved: {summary_path}")
    print("✅ Pre-training completed!")



if __name__ == "__main__":
    main()

🚀 Simple Sensor Pre-Training
Loading data...
Original data shape: (3627, 50, 12)
Training data shape: (2627, 50, 12)
Will train 12 sensors

🏋️ Training 12 LSTM Autoencoders...

Training sensor 0...
Data shape: (2627, 50, 1)
Train: 2101, Val: 526
Starting training with checkpointing...
Epoch 1/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.4507
Epoch 1: val_loss improved from inf to 0.06568, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step - loss: 0.4485 - val_loss: 0.0657 - learning_rate: 0.0010
Epoch 2/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.1605
Epoch 2: val_loss improved from 0.06568 to 0.05073, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.1605 - val_loss: 0.0507 - learning_rate: 0.0010
Epoch 3/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.1742
Epoch 3: val_loss did not improve from 0.05073
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.1739 - val_loss: 0.0551 - learning_rate: 0.0010
Epoch 4/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1406
Epoch 4: val_loss did not improve from 0.05073
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.1405 - val_loss: 0.0510 - learning_rate: 0.0010
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1340
Epoch 5: val_loss improved from 0.05073 to 0.04698, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 0.1339 - val_loss: 0.0470 - learning_rate: 0.0010
Epoch 6/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.1198
Epoch 6: val_loss improved from 0.04698 to 0.04514, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.1197 - val_loss: 0.0451 - learning_rate: 0.0010
Epoch 7/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.1132
Epoch 7: val_loss improved from 0.04514 to 0.04402, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_0_best.h5


...

66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 0.1544 - val_loss: 0.0677 - learning_rate: 0.0010
Epoch 3/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1159
Epoch 3: val_loss improved from 0.06773 to 0.06059, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_11_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.1158 - val_loss: 0.0606 - learning_rate: 0.0010
Epoch 4/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.1097
Epoch 4: val_loss did not improve from 0.06059
66/66 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - loss: 0.1096 - val_loss: 0.0610 - learning_rate: 0.0010
Epoch 5/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1121
Epoch 5: val_loss did not improve from 0.06059
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.1121 - val_loss: 0.0620 - learning_rate: 0.0010
Epoch 6/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1040
Epoch 6: val_loss improved from 0.06059 to 0.05925, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_11_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - loss: 0.1041 - val_loss: 0.0593 - learning_rate: 0.0010
Epoch 7/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.1073
Epoch 7: val_loss improved from 0.05925 to 0.05835, saving model to /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_11_best.h5


66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.1072 - val_loss: 0.0583 - learning_rate: 0.0010
Epoch 8/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1297
Epoch 8: val_loss did not improve from 0.05835
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 0.1298 - val_loss: 0.0703 - learning_rate: 0.0010
Epoch 9/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1140
Epoch 9: val_loss did not improve from 0.05835
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.1140 - val_loss: 0.0615 - learning_rate: 0.0010
Epoch 10/100
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0992
Epoch 10: val_loss did not improve from 0.05835
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0993 - val_loss: 0.0590 - learning_rate: 0.0010
Epoch 11/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0996
Epoch 11: val_loss did not improve from 0.05835
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - loss: 0.0998 - val_loss: 0.0590 - learning_rate: 0.0010
Epoch 12/100
65/66 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/st

✅ Sensor 11 completed
   Final model: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/sensor_11_model.h5
   Best checkpoint: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/checkpoints/sensor_11_best.h5
   Baseline error: 0.058348 ± 0.095469

📊 TRAINING SUMMARY
✅ Successful: 12/12
💾 Models saved to: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM//sensor/model/

🏆 Successfully trained sensors:
  Sensor 0: baseline error 0.021486
  Sensor 1: baseline error 0.014052
  Sensor 2: baseline error 0.034632
  Sensor 3: baseline error 0.001114
  Sensor 4: baseline error 0.012735
  Sensor 5: baseline error 0.004379
  Sensor 6: baseline error 0.052594
  Sensor 7: baseline error 0.079883
  Sensor 8: baseline error 0.049380
  Sensor 9: baseline error 0.035827
  Sensor 10: baseline error 0.042994
  Sensor 11: baseline error 0.058348

💾 Summary saved: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/sensor/model/training_summary.pkl
✅ Pre-training completed!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')